In [ ]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from tqdm.notebook import tqdm
import seaborn as sns
import time
import pickle
import cv2
from sklearn.preprocessing import LabelBinarizer
from IPython.display import Image
from PIL import Image

## Set Up
Consider a 2-dimensional data set in which all points with $x_1 > x_2$ belong to the positive class, and all points with x_1 ≤ x_2 belong to the negative class. Therefore, the true separator of the two classes is linear hyperplane (line) defined by $x_1 − x_2 = 0$. Now create a training data set with 10 points randomly generated inside the unit square in the positive quadrant. Label each point depending on whether or not the first coordinate x1 is greater than its second coordinate $x_2$. Now consider the following loss function for training pair $(\bar{X} ,y)$ and weight vector $\tilde{W}$:
$$L = \mathrm{max}\{0, a-y(\bar{W} \cdot \bar{X})\},$$

where the test instances are predicted as $\hat{y} = sign{\bar{W}\cdot  \bar{X}}$. For this problem, $\bar{W}  = [w1,w2], \bar{X } = [x1,x2]$ and $\hat{y} = \mathrm{sign}(w1x1 + w2x2)$. A value of a = 0 corresponds to the perceptron criterion and a value of $a = 1$ corresponds to hinge-loss.

## 1. 
Implement the perceptron algorithm without regularization, train it on the 10 points above, and test its accuracy on 5000 randomly generated points inside the unit square. Generate the test points using the same procedure as the training points. You need to have your own implementation of the perceptron algorithm.

In [ ]:
np.random.seed(0)
X = np.random.random((10,2))
X_test = np.random.random((5000,2))

In [ ]:
def generate_Y(X):
    Y = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        if X[i,0]> X[i,1]:
            Y[i] = 1
        else:
            Y[i] = -1
    return Y

In [ ]:
Y = generate_Y(X)
Y_test = generate_Y(X_test)


In [ ]:
def predict_y(W, X):
    return np.sign(X@W)

In [ ]:
def train_perceptron(X, Y, loss, lr, stopping_criteria = 20000):
    W = np.zeros(X.shape[1])
    b = 0
    y_hat = np.zeros(X.shape[0])

    if loss == 'perceptron_criterion':
        while len(np.where(Y!=y_hat)[0])>0:
            for i, x_i in enumerate(X):
                y_hat[i] = predict_y(W, x_i)
                if y_hat[i]!=Y[i]:
                  W += lr*x_i*Y[i]

    elif loss == 'hinge':
        #Add in stopping criterion since hinge loss isn't gauranteed to converge
        incorrect_tracker = np.zeros(len(X))
        while len(np.where(np.repeat(1, len(X)) - Y*(X@W) > 0)[0]):
            incorrect_count = len(np.where(np.repeat(1, len(X)) - Y*(X@W) > 0)[0])-1
            incorrect_tracker[incorrect_count]+=1
            if incorrect_tracker[incorrect_count]> stopping_criteria:
              break
            else:
              
              for i, x_i in enumerate(X):
                  y_hat[i] = predict_y(W, x_i)
                  if 1 - (x_i@W)*Y[i] > 0:
                      W += lr*x_i*Y[i]
    return W


In [ ]:
def eval_perceptron(W, X, Y):
    Y_hat = np.sign(X@W)
    acc = np.sum(Y_hat == Y)/len(Y)
    return acc

In [ ]:
W = train_perceptron(X, Y, loss = 'perceptron_criterion',lr = 0.1)
test_acc = eval_perceptron(W, X_test, Y_test)
print(W)
print("Test accuracy with perceptron Criterion", test_acc)

In [ ]:
plt.scatter(X[:,0][np.argwhere(Y>0)], X[:,1][np.argwhere(Y>0)])
plt.scatter(X[:,0][np.argwhere(Y<=0)], X[:,1][np.argwhere(Y<=0)])

## 2.
Change the perceptron criterion to hinge-loss in your implementation for training, and repeat the accuracy computation on the same test points above. Regularization is not used. In which case do you obtain better accuracy and why?

In [ ]:
W_hinge = train_perceptron(X, Y, loss = 'hinge', lr = 0.01)
test_acc_hinge = eval_perceptron(W_hinge, X_test, Y_test)
print(W_hinge)
print("Test aaccuraccy with hinge loss:",test_acc_hinge)

Hinge loss gets better accuracy on the test set. This is because perceptron has infinitely many ways it can draw a separating hyperplane between the classes while hinge loss has the 1 margin it has to account for. The diagonal line where x = y is a natural split for this dataset that will always classify all point correctly. Because of the margin hinge loss is more likely to be forced closer to that true middle line whereas with the regular perceptron criterion there can be more variance.


## 3. 
In which case do you think that the classification of the same 5000 test instances will not change significantly by using a different set of 10 training points?


In [ ]:
## Try different iterations of train set to test answer
percep_higher = 0
hinge_higher = 0
for i in range(30):
    np.random.seed(i)
    print(f"ITER {i}")
    X_train = np.random.random((10,2))
    Y_train = generate_Y(X_train)
    W_percep_crit = train_perceptron(X_train, Y_train, loss = 'perceptron_criterion',lr = 0.01)
    test_acc_percep_crit = eval_perceptron(W_percep_crit, X_test, Y_test)
    print(f"Accuracy w/ perceptron criterion: {test_acc_percep_crit}")

    W_hinge_testing = train_perceptron(X_train, Y_train, loss = 'hinge',lr = 0.01)
    test_acc_hinge = eval_perceptron(W_hinge_testing, X_test, Y_test)
    print(f"Accuracy w/ hinge loss: {test_acc_hinge}")

    if test_acc_percep_crit > test_acc_hinge:
        percep_higher +=1
    else:
        hinge_higher+=1
print(percep_higher)
print(hinge_higher)

Accuracy with the perceptron criteriion will change more because there are infinitely many solutions that will separate the classes and the location can be more random depending on the training data/ order in which the algorithm goes through the training data. This could mean that some of the separating hyperplanes found in training with the perceptron criterion could be farther from the true middle line and thus perform worse on the test set. Hinge loss, because of the margin, does a better job of pushing the separating hyperplane closer to the middle. We see below in a simulation that sometimes accuracy will be worse when trained with hinge loss but perceptron criterion is worse for a majority of 30 random test cases tried below. However, I will note that hinge loss does do worse than perceptron a decent amount of the time.